In [46]:

import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt

import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.utils import to_categorical
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

path_train = "/Users/fabriziovasquez/Downloads/PFCII/chest_2.csv"
path_test = "/Users/fabriziovasquez/Downloads/PFCII/chest_3.csv"

In [55]:
# Verificar si existen saltos de datos
# Reemplazos de NaN's

def get_window_labels(path):
  df = pd.read_csv(path)
  df = df[df['mode'] < 4]
  #state = np.repeat(['base', 'stress', 'amusement'], df.shape[0])

  window_size = 1400  # 2 segundos * 700 muestras/segundo
  step_size = 700  # 50% de superposición

  windows = []
  labels = []

  df['c_eda_mean'] = df['c_eda'].rolling(window=window_size).mean()
  df['c_eda_std'] = df['c_eda'].rolling(window=window_size).std()
  
  df = df.interpolate()
  
  df = df.iloc[window_size - 1:]
  
  for s in df['mode'].unique():
      # data = df[df['mode'] == s]
      for i in range(0, len(df) - window_size, step_size):
        window = df[['c_eda_mean', 'c_eda_std']].iloc[i:i+window_size].values
        windows.append(window)
        label = df['mode'].iloc[i]
        labels.append(label)


  return windows, labels


In [23]:
#windows, labels = get_window_labels(path)

In [39]:
windows[2]

array([[7.23444012, 0.02683003],
       [7.23446955, 0.02684146],
       [7.23449707, 0.02684918],
       ...,
       [7.23994909, 0.03531768],
       [7.23989514, 0.0353515 ],
       [7.23984064, 0.03538554]])

In [56]:
train_windows = []
train_labels = []

windows, labels = get_window_labels(path)
train_windows.extend(windows)
train_labels.extend(labels)

test_windows = []
test_labels = []

windows, labels = get_window_labels(path_test)
test_windows.extend(windows)
test_labels.extend(labels)


[array([[7.23441669, 0.0268259 ],
        [7.23444012, 0.02683003],
        [7.23446955, 0.02684146],
        ...,
        [7.24000359, 0.03528458],
        [7.23994909, 0.03531768],
        [7.23989514, 0.0353515 ]]),
 array([[7.23496028, 0.02532018],
        [7.23499053, 0.02534773],
        [7.2350175 , 0.02537143],
        ...,
        [7.17895099, 0.06176218],
        [7.17880522, 0.06176713],
        [7.17865944, 0.06177624]]),
 array([[7.23984064, 0.03538554],
        [7.23978751, 0.03541776],
        [7.23972593, 0.03544801],
        ...,
        [7.11858831, 0.05458762],
        [7.11862319, 0.05464272],
        [7.1186567 , 0.05469477]]),
 array([[7.17851067, 0.06178749],
        [7.17836108, 0.06179704],
        [7.17821421, 0.06180879],
        ...,
        [7.22636196, 0.13664502],
        [7.22662817, 0.13671244],
        [7.22690255, 0.13678901]]),
 array([[7.11869349, 0.0547531 ],
        [7.11873055, 0.05481294],
        [7.11876624, 0.05486863],
        ...,
        [

In [63]:
train_windows = np.array(train_windows)
train_labels = np.array(train_labels)
test_windows = np.array(test_windows)
test_labels = np.array(test_labels)

train_windows_flat = train_windows.reshape(train_windows.shape[0], -1)
test_windows_flat = test_windows.reshape(test_windows.shape[0], -1)

model = RandomForestClassifier(n_estimators=500)
model.fit(train_windows_flat, train_labels)

RandomForestClassifier(n_estimators=500)

In [64]:
test_preds = model.predict(test_windows_flat)
print(classification_report(test_labels, test_preds))


              precision    recall  f1-score   support

           1       0.53      1.00      0.69      3417
           2       0.00      0.00      0.00      1920
           3       0.00      0.00      0.00      1119

    accuracy                           0.53      6456
   macro avg       0.18      0.33      0.23      6456
weighted avg       0.28      0.53      0.37      6456



/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/homebrew/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
